## Define LLMs

In [73]:
# OpenAI
import openai
from semantic_router.utils.logger import logger

# Docs # https://platform.openai.com/docs/guides/function-calling
def llm_openai(prompt: str, model: str = "gpt-4") -> str:
    try:
        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": f"{prompt}"},
            ],
        )
        ai_message = response.choices[0].message.content
        if not ai_message:
            raise Exception("AI message is empty", ai_message)
        logger.info(f"AI message: {ai_message}")
        return ai_message
    except Exception as e:
        raise Exception("Failed to call OpenAI API", e)

In [102]:
# Mistral
import os
import requests

# Docs https://huggingface.co/docs/transformers/main_classes/text_generation
HF_API_TOKEN = os.environ["HF_API_TOKEN"]

def llm_mistral(prompt: str) -> str:
    api_url = "https://z5t4cuhg21uxfmc3.us-east-1.aws.endpoints.huggingface.cloud/"
    headers = {
        "Authorization": f"Bearer {HF_API_TOKEN}",
        "Content-Type": "application/json",
    }

    response = requests.post(
        api_url,
        headers=headers,
        json={
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 1000,
                "temperature": 0.2,
            },
        },
    )
    if response.status_code != 200:
        raise Exception("Failed to call HuggingFace API", response.text)

    ai_message = response.json()[0]['generated_text']
    if not ai_message:
            raise Exception("AI message is empty", ai_message)
    logger.info(f"AI message: {ai_message}")
    return ai_message

### Now we need to generate config from function specification using LLM

In [134]:
import json

from pydantic import BaseModel
from semantic_router.utils.logger import logger

def generate_config(schema: dict) -> dict:
    logger.info("Generating config...")

    class GetWeatherSchema(BaseModel):
        location: str

        class Config:
            name = "get_weather"

    example_schema = GetWeatherSchema.schema()

    example_config = {
        "name": "get_weather",
        "utterances": [
            "What is the weather like in SF?",
            "What is the weather in Cyprus?",
            "weather in London?",
            "Tell me the weather in New York",
            "what is the current weather in Paris?",
        ],
    }

    prompt = f"""
    Given the following Pydantic function schema,
    generate a config ONLY in a valid JSON format.
    For example:
    SCHEMA: {example_schema}
    CONFIG: {example_config}


    GIVEN SCHEMA: {schema}
    GENERATED CONFIG: <generated_response_in_json>
    """

    ai_message = llm_openai(prompt)
    print(f"AI message: {ai_message}")

    # Parsing for Mistral model
    ai_message = ai_message.replace("CONFIG:", "").replace("'", '"').strip()

    try:
        route_config = json.loads(ai_message)
        logger.info(f"Generated config: {route_config}")
        return route_config
    except json.JSONDecodeError as json_error:
        logger.error(f"JSON parsing error {json_error}")
        print(f"AI message: {ai_message}")
        return {}

Extract function parameters using `Mistral` open-source model

In [138]:
def extract_parameters(query: str, schema: dict) -> dict:
    logger.info("Extracting parameters...")
    example_query = "what is the weather in London?"

    class GetWeatherSchema(BaseModel):
        location: str

        class Config:
            name = "get_weather"

    example_schema = GetWeatherSchema.schema()

    example_parameters = {
        "location": "London",
    }

    prompt = f"""
    Given the following function schema and query, extract the parameters from the
    query, in a valid JSON format.
    Example:
    SCHEMA:
    {example_schema}
    QUERY:
    {example_query}
    PARAMETERS:
    {example_parameters}
    GIVEN SCHEMA:
    {schema}
    GIVEN QUERY:
    {query}
    EXTRACTED PARAMETERS:
    """

    ai_message = llm_openai(prompt)

    ai_message = ai_message.replace("'", '"').strip()

    try:
        parameters = json.loads(ai_message)
        logger.info(f"Extracted parameters: {parameters}")
        return parameters
    except json.JSONDecodeError as json_error:
        logger.error(f"JSON parsing error {json_error}")
        return {}

Set up the routing layer

In [139]:
from semantic_router.schema import Route
from semantic_router.encoders import CohereEncoder
from semantic_router.layer import RouteLayer
from semantic_router.utils.logger import logger


def get_route_layer(config: list[dict]) -> RouteLayer:
    logger.info("Getting route layer...")
    encoder = CohereEncoder()

    routes = []
    print(f"Config: {config}")
    for route in config:
        if "name" in route and "utterances" in route:
            print(f"Route: {route}")
            routes.append(Route(name=route["name"], utterances=route["utterances"]))
        else:
            logger.warning(f"Misconfigured route: {route}")

    return RouteLayer(encoder=encoder, routes=routes)

### Workflow

In [140]:
from pydantic import BaseModel

class GetTimeSchema(BaseModel):
    location: str

    class Config:
        name = "get_time"

get_time_schema = GetTimeSchema.schema()

def get_time(location: str) -> str:
    # Validate parameters
    GetTimeSchema(location=location)

    print(f"Calling get_time function with location: {location}")
    return "get_time"


route_config = generate_config(get_time_schema)
route_layer = get_route_layer([route_config])

queries = [
    "What is the weather like in Barcelona?",
    "What time is it in Taiwan?",
    "What is happening in the world?",
    "what is the time in Kaunas?",
    "Im bored",
    "I want to play a game",
    "Banana",
]

# Calling functions
for query in queries:
    function_name = route_layer(query)
    print(function_name, query)

    if function_name == "get_time":
        function_parameters = extract_parameters(query, get_time_schema)
        try:
            # Call the function
            get_time(**function_parameters)
        except ValueError as e:
            logger.error(f"Error: {e}")

2023-12-14 17:28:22 INFO semantic_router.utils.logger Generating config...
2023-12-14 17:28:28 INFO semantic_router.utils.logger AI message: {"name": "get_time", "utterances": ["What is the time in SF?", "What is the current time in London?", "Time in Tokyo?", "Tell me the time in New York", "What is the time now in Paris?"]}
2023-12-14 17:28:28 INFO semantic_router.utils.logger Generated config: {'name': 'get_time', 'utterances': ['What is the time in SF?', 'What is the current time in London?', 'Time in Tokyo?', 'Tell me the time in New York', 'What is the time now in Paris?']}
2023-12-14 17:28:28 INFO semantic_router.utils.logger Getting route layer...


AI message: {"name": "get_time", "utterances": ["What is the time in SF?", "What is the current time in London?", "Time in Tokyo?", "Tell me the time in New York", "What is the time now in Paris?"]}
Config: [{'name': 'get_time', 'utterances': ['What is the time in SF?', 'What is the current time in London?', 'Time in Tokyo?', 'Tell me the time in New York', 'What is the time now in Paris?']}]
Route: {'name': 'get_time', 'utterances': ['What is the time in SF?', 'What is the current time in London?', 'Time in Tokyo?', 'Tell me the time in New York', 'What is the time now in Paris?']}
None What is the weather like in Barcelona?


2023-12-14 17:28:29 INFO semantic_router.utils.logger Extracting parameters...


get_time What time is it in Taiwan?
